In [1]:
import getpass
username = getpass.getuser()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/{:}/GitHub/mouse_bandit/data_preprocessing_code'.format(username))
sys.path.append('/Users/{:}/GitHub/mouse_bandit'.format(username))
import support_functions as sf

sys.path.append('/Users/{:}/GitHub/mouse_bandit/jupyter_notebooks/helper_functions'.format(username))
import bandit_modeling as bm

import sklearn
from sklearn.model_selection import train_test_split

% matplotlib inline

Markov process is one in which the future state of the system does not depend on more (history) than the current state

In [2]:
# function inputs: DATA, EMISSION_PROB, TRANSITION_PROB=0.02, STRATEGY, N_PREV_TRIALS
def hmm_predict(data, emission_prob, transition_prob=0.02, strategy='greedy', n_prev_trials=10):
    
    '''
    Inputs:
        data = pandas dataframe containing port and reward features
        emission_prob = reward probability observed by the agent
        transition_prob = probability of transitioning between markovian states
        strategy = 'greedy,' 'thompson,' 'eps_greedy'
        n_prev_trials = int 1 to 10, how many previous trials are used to calculate belief state
    '''
    
    # latent state z (0 left, 1 right)
    # action a (0 left, 1 right)
    # reward r (0 no reward, 1 reward)
    # transition_prob (0 to 1)

    # emission probabilities (observed probabilities of reward)
    p = emission_prob # probability of reward for correct side
    q = 1-p # probability of reward for incorrect side
    s = 1-transition_prob # probability of remaining in same state

    # transition matrix T[i,j] = p(i->j)
    T = np.array([[s, transition_prob],
                  [transition_prob, s]])

    # observation array O
    # O[r,z,a] = Pr(reward=r | state=z, action=a)
    O = np.zeros((2,2,2)) # each component has two conditions 0,1
    
    # right choice = 0, no reward = 0, right state = 0
    O[:,:,0] = np.array([[1-p, 1-q],[p,q]]) 
    O[:,:,1] = np.array([[1-q, 1-p],[q,p]])

    # split data into train and test sets (ultimately using same seeds as with LR?)

    n_trials = data.shape[0]

    #include n_prev_trials past choice and reward values (this is most convenient given the current data structure)
    port_features = []
    reward_features = []
    for col in data:
        if '_Port' in col:
            port_features.append(col)
        elif '_Reward' in col:
            reward_features.append(col)

    y_predict = np.zeros(n_trials)
    beliefs = np.zeros(n_trials)

    for trial in range(n_trials):
        curr_trial = data.iloc[trial]
        actions = curr_trial[port_features].values.astype('int')
        rewards = curr_trial[reward_features].values.astype('int')
        beliefs_curr = np.nan*np.ones((n_prev_trials+1,2))
        beliefs_curr[0] = [0.5,0.5] # initial belief is equal for each port

        for i in range(n_prev_trials):

            assert np.allclose(beliefs_curr[i].sum(), 1.0), "Beliefs must sum to one!"

            belief_temp = O[rewards[i],:,actions[i]] * beliefs_curr[i]

            beliefs_curr[i+1] = T.dot(belief_temp) # take dot product of transition matrix and previous belief

            beliefs_curr[i+1] /= beliefs_curr[i+1].sum()
        
        if strategy == 'greedy':
            y_predict[trial] = np.where(beliefs_curr[-1] == beliefs_curr[-1].max())[0][0] 
            
        elif strategy == 'thompson':
            y_predict[trial] = np.random.choice(2,p=beliefs_curr[-1])
            #y_predict[trial] = np.random.choice(2,p=[p,q])
        
        elif strategy == 'eps_greedy':
            print('need to work on this one')
        
        beliefs[trial] = beliefs_curr[-1][1]
    
    return y_predict, beliefs

In [3]:
data = pd.read_csv('/Users/{:}/Dropbox (HMS)/mouse_bandit/markov_full.csv'.format(username), index_col=0)
#data = data[data['Mouse ID']=='Baby']
data = data[data['Condition']=='80-20']

In [4]:
data_train, data_test = train_test_split(data, test_size=0.3, random_state=1) # do this to match the logreg dataset

In [5]:
y_predict, beliefs = hmm_predict(data, 0.8, transition_prob=0.02)

In [ ]:
# for sending out prediction to test against real data
#y_predict = np.array(y_predict)
#np.savetxt('/Users/celiaberon/Dropbox (HMS)/mouse_bandit/model_outputs/hmm_8020_y_predict.csv', y_predict, delimiter=',')

In [ ]:
'''Greedy model'''

decision = data_test['Decision'].values
prev_decision = data_test['1_Port'].values

switch = np.abs(decision-prev_decision)
switch_predict = np.abs(y_predict-prev_decision)

accuracy_greedy = np.mean(y_predict==decision)
#accuracy=1-np.abs([y_predict[i]-decision[i] for i in range(len(beliefs))]).sum()/len(beliefs)
print('accuracy = ', accuracy_greedy)

precision=1-np.abs([switch_predict[i]-switch[i] for i in np.where(switch_predict==1)]).sum()/np.sum(switch_predict==1)
print('precision =', precision)

recall=1-np.abs([switch_predict[i]-switch[i] for i in np.where(switch==1)]).sum()/np.sum(switch==1)
print('recall =', recall)

acc_greedy_switch,acc_greedy_stay,F1=sf.score_both_and_confuse(switch_predict,switch,confusion=False,disp=True)

np.random.shuffle(switch.values)
shuffled_greedy_switch = sf.score_both_and_confuse(switch_predict,switch,confusion=False,disp=True)

In [ ]:
y_predict_thompson, beliefs = hmm_predict(data_test, 0.75, transition_prob=0.035, strategy='thompson')

In [ ]:
'''With Thompson sampling'''
y_predict = [np.random.choice(2,p=[1-beliefs[i],beliefs[i]]) for i in range(len(beliefs))]

decision = data_test['Decision'].values
prev_decision = data_test['1_Port'].values

switch = np.abs(decision-prev_decision)
switch_predict = np.abs(y_predict-prev_decision)

accuracy_thom = np.mean(y_predict==decision)
#accuracy=1-np.abs([y_predict[i]-decision[i] for i in range(len(beliefs))]).sum()/len(beliefs)
print('accuracy = ', accuracy_thom)

precision=1-np.abs([switch_predict[i]-switch[i] for i in np.where(switch_predict==1)]).sum()/np.sum(switch_predict==1)
print('precision =', precision)

recall=1-np.abs([switch_predict[i]-switch[i] for i in np.where(switch==1)]).sum()/np.sum(switch==1)
print('recall =', recall)

acc_thom_switch,acc_thom_stay,F1=sf.score_both_and_confuse(switch_predict,switch,confusion=False,disp=True)

np.random.shuffle(switch.values)
shuffled_thom_switch = sf.score_both_and_confuse(switch_predict,switch,confusion=False,disp=True)

In [ ]:
decision = data_test['Decision'].values
prev_decision = data_test['1_Port'].values
switch = np.abs(decision-prev_decision)

acc_thom_full = []
acc_switch_full = []
acc_stay_full = []

for n in range(0,100):
    y_predict = [np.random.choice(2,p=[1-beliefs[i],beliefs[i]]) for i in range(len(beliefs))]

    switch_predict = np.abs(y_predict-prev_decision)

    accuracy_thom = np.mean(y_predict==decision)

    precision=1-np.abs([switch_predict[i]-switch[i] for i in np.where(switch_predict==1)]).sum()/np.sum(switch_predict==1)

    recall=1-np.abs([switch_predict[i]-switch[i] for i in np.where(switch==1)]).sum()/np.sum(switch==1)

    acc_thom_switch,acc_thom_stay,F1=sf.score_both_and_confuse(switch_predict,switch,confusion=False,disp=True)
    
    acc_thom_full.append(accuracy_thom)
    acc_stay_full.append(acc_thom_stay)
    acc_switch_full.append(acc_thom_switch)

In [ ]:
np.mean(beliefs) # should be 0.5 if belief switches between ports.

In [ ]:

barWidth = 0.2
# The x position of bars
r1 = np.arange(len(height_b))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

conditions = ['full', 'stay', 'switch']
plt.bar(r1, height_9010_greedy, width=barWidth, label='90-10')
plt.bar(r2, height_8020_greedy, width=barWidth, label='80-20')
plt.bar(r3, height_7030_greedy, width=barWidth, label='70-30')

plt.xticks(r2, conditions)
plt.ylabel('accuracy')
plt.legend()
plt.ylim((0,0.95))
plt.title('HMM Greedy')

In [ ]:
barWidth = 0.2
# The x position of bars
r1 = np.arange(len(height_b))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

conditions = ['full', 'stay', 'switch']
plt.bar(r1, height_9010_thom, width=barWidth, label='90-10')
plt.bar(r2, height_8020_thom, width=barWidth, label='80-20')
plt.bar(r3, height_7030_thom, width=barWidth, label='70-30')


plt.xticks(range(len(height_b)), conditions)
plt.ylabel('accuracy')
plt.legend()
plt.ylim((0,0.95))
plt.title('HMM Thompson sampling')

In [ ]:
acc_thom_stay = acc_stay_full.copy()
acc_thom_switch = acc_switch_full.copy()
accuracy_thom = acc_thom_full.copy()

In [ ]:
height_a = [accuracy_greedy, acc_greedy_stay, acc_greedy_switch]
height_b = [np.mean(accuracy_thom), np.mean(acc_thom_stay), np.mean(acc_thom_switch)]

ystd2 = [np.std(accuracy_thom), np.std(acc_thom_stay), np.std(acc_thom_switch)]
yerr2 = [ystd2[i] / np.sqrt(100) for i in range(len(ystd2))]

barWidth = 0.2
# The x position of bars
r1 = np.arange(len(height_b))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

conditions = ['full', 'stay', 'switch']
plt.bar(r1, height_a, width=barWidth, label='hmm_greedy')
plt.bar(r2, height_b, width=barWidth, yerr=yerr2, capsize=3, label='hmm_thompson')

plt.xticks([0.1, 1.1, 2.1], conditions, size=15)
plt.ylabel('accuracy', size=15)
plt.legend()

In [ ]:
data.head()

In [ ]:
# take just one mouse, session
plt.figure(figsize=(15,5))

plt.subplot(3,1,1)
plt.plot(data['Higher p port'].values[0:500])
plt.xlabel('trial')
plt.ylabel('hidden state')

plt.subplot(3,1,2)
plt.plot(beliefs[0:500])
plt.ylabel('belief from HMM')

plt.subplot(3,1,3)
plt.plot(data['Decision'].values[0:500])
plt.ylabel('mouse choice')
plt.xlabel('Trial', size=20)

In [ ]:
X, y = bm.Xy_history(data)
y_predict = pd.Series(y_predict, index=y.index)
reward_combos_hmm, p_switch_hmm = bm.sequences_predict_switch(X,y_predict)
reward_combos_true, p_switch_true = bm.sequences_predict_switch(X,y)

In [ ]:
p_switch_hmm-p_switch_true

In [ ]:
plt.bar(left=np.arange(len(reward_combos)) ,height=p_switch_true-p_switch_hmm)